This is a copy of Benjamin Sorensen's blog post about using redist, which you can find here: https://www.benjaminsorensen.me/post/mt-redistricting/

In [1]:
library(sf)
library(ggplot2)
library(tidyverse)
library(tibble)
library(magrittr)
library(redist)
library(sp)
library(spData)
library(spdep)
library(igraph)

Linking to GEOS 3.8.0, GDAL 3.0.4, PROJ 6.3.1

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tibble  3.1.0     v dplyr   1.0.5
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1
v purrr   0.3.4     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract


To access larger datasets in this package, install the spDataLarge
package with: `install.packages('spDataLarge',
repos='https://nowosad.github.io/drat/', type='source')`


Attaching package: 'igraph'


The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union


The following objects are masked from

In [2]:
va_2015_file <- "C:/Users/madie/OneDrive/data/pre-redist/VA_precinct_2015/VA_precinct_2015.shp"
df <- st_read(va_2015_file)

Reading layer `VA_precinct_2015' from data source `C:\Users\madie\OneDrive\data\pre-redist\VA_precinct_2015\VA_precinct_2015.shp' using driver `ESRI Shapefile'
Simple feature collection with 2577 features and 14 fields
geometry type:  MULTIPOLYGON
dimension:      XYZ
bbox:           xmin: 9963623 ymin: 6151696 xmax: 12442960 ymax: 7217011
z_range:        zmin: 0 zmax: 0
projected CRS:  NAD83 / Virginia North (ftUS)


In [3]:
adjlist <- spdep::poly2nb(df, queen = FALSE)

In [4]:
# much faster for st_distance to use centroids rather than polygons
distmat <- 
  df %>% 
  mutate(centroid = st_centroid(geometry)) %>% 
  st_set_geometry(value = "centroid") %>% 
  select(-geometry) %>% 
  st_distance()

In [6]:
start.time <- Sys.time()
alg_mcmc <- redist.mcmc(
  adjobj = adjlist,
  popvec = df$pop,
#  initcds = initcds$cd_ns, I dont' have initial districts I'm starting with
  maxiterrsg = 1e8,
  ndists = 11,
  nsims = 10000,
  constraint = c("population", "compact"),
  constraintweights = c(1, 1),
  popcons = .1,
  ssdmat = distmat,
  savename = "redist.mcmc.2015"
) %>% 
  invisible()
end.time <- Sys.time()
print(end.time - start.time)


redist.mcmc(): Automated Redistricting Simulation Using
         Markov Chain Monte Carlo

Preprocessing data.


Using redist.rsg() to generate starting values.

10 percent done.
Metropolis acceptance ratio: 0.317317

20 percent done.
Metropolis acceptance ratio: 0.235118

30 percent done.
Metropolis acceptance ratio: 0.198066

40 percent done.
Metropolis acceptance ratio: 0.175794

50 percent done.
Metropolis acceptance ratio: 0.156631

60 percent done.
Metropolis acceptance ratio: 0.13969

70 percent done.
Metropolis acceptance ratio: 0.128304

80 percent done.
Metropolis acceptance ratio: 0.12089

90 percent done.
Metropolis acceptance ratio: 0.112124

100 percent done.
Metropolis acceptance ratio: 0.106511

Time difference of 33.45871 mins
